In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob

In [2]:
#read in kaggle files: https://www.kaggle.com/snapcrack/all-the-news
df = pd.read_csv('../datasets/articles1.csv')
df1 = pd.read_csv('../datasets/articles2.csv')
df2 = pd.read_csv('../datasets/articles3.csv')

Concat/union all the dataframes together, reset/clean the index

In [5]:
df = pd.concat([df, df1, df2]).reset_index()
#cleaning the index
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.drop(['index'], axis = 1, inplace = True)
df.drop(['id'], axis = 1, inplace = True)

#ensure key fields are the data types I want
df[["title", "publication", "year"]] = df[["title", "publication","year"]].astype(str) 
df.tail()

/Users/erikrood/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,author,content,date,month,publication,title,url,year
235135,Avi Selk,Uber driver Keith Avila picked up a p...,2016-12-30,12.0,Washington Post,An eavesdropping Uber driver saved his 16-year...,https://web.archive.org/web/20161231004909/htt...,2016.0
235136,Sarah Larimer,Crews on Friday continued to search L...,2016-12-30,12.0,Washington Post,Plane carrying six people returning from a Cav...,https://web.archive.org/web/20161231004909/htt...,2016.0
235137,Renae Merle,When the Obama administration announced a...,2016-12-30,12.0,Washington Post,After helping a fraction of homeowners expecte...,https://web.archive.org/web/20161231004909/htt...,2016.0
235138,Chelsea Harvey,This story has been updated. A new law in...,2016-12-30,12.0,Washington Post,"Yes, this is real: Michigan just banned bannin...",https://web.archive.org/web/20161231004909/htt...,2016.0
235139,Christopher Ingraham,The nation’s first recreational marijuana...,2016-12-29,12.0,Washington Post,What happened in Washington state after voters...,https://web.archive.org/web/20161231004909/htt...,2016.0


In [91]:
#filter to 2016-2017, only years with reasonable n-counts across all publications
years = ['2016.0','2017.0']
df = df.loc[df['year'].isin(years)]

Perform sentiment analysis on base dataset

In [93]:
#used to analyze sentiment w/ textblob, bucketing into positive/neutral/negative
def analize_sentiment(headline):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(headline)
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1
    
#used to analyze sentiment w/ textblob, leaving the raw output
def analize_sentiment_raw(headline):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(headline)
    #polarity = positive/negative
    return analysis.sentiment.polarity 


#used to analyze sentiment w/ textblob, leaving the raw output
def analize_sentiment_raw_subjectivity(headline):
    '''
    Utility function to classify the subjectivity of a tweet
    using textblob.
    '''
    analysis = TextBlob(headline)
    return analysis.sentiment.subjectivity 

In [94]:
#select function and feed in here
df['SA'] = np.array([ analize_sentiment_raw(headline) for headline in df['title']])

In [95]:
#export this because it takes forever to process the above, we can then read in an manipulate as we want
df.to_csv('../datasets/news_project/all_news_sentiment_raw_polarity.csv')

### Read in CSV to analyze, after performing sentiment analysis above

In [32]:
df = pd.read_csv('../datasets/news_project/all_news_sentiment_raw_polarity.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.head()

,title,publication,author,date,year,month,url,content,SA
0,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...,0.318182
1,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood...",0.136364
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri...",0.136364
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t...",-0.031818
4,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ...",0.136364


In [33]:
#extract clean year from date
df[["date"]] = df[["date"]].astype(str) 
df['publish_year'] = df['date'].str[:4]
#filter to 2016, 2017
years = ['2016','2017']
df = df.loc[df['publish_year'].isin(years)]
#NY times looks wonky (skewed positive, dropping from exercise)
df = df.loc[df['publication'] != 'New York Times']

In [34]:
df.tail()

,title,publication,author,date,year,month,url,content,SA,publish_year
135804,An eavesdropping Uber driver saved his 16-year...,Washington Post,Avi Selk,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Uber driver Keith Avila picked up a p...,0.00,2016
135805,Plane carrying six people returning from a Cav...,Washington Post,Sarah Larimer,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Crews on Friday continued to search L...,-0.40,2016
135806,After helping a fraction of homeowners expecte...,Washington Post,Renae Merle,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,When the Obama administration announced a...,-0.05,2016
135807,"Yes, this is real: Michigan just banned bannin...",Washington Post,Chelsea Harvey,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,This story has been updated. A new law in...,0.20,2016
135808,What happened in Washington state after voters...,Washington Post,Christopher Ingraham,2016-12-29,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,The nation’s first recreational marijuana...,0.00,2016


In [35]:
#slice the df to limit to just sentiment analysis score, publication
df_clean = df[['SA', 'publication']]

### Pivot to get avg sentiment by publication - polarity

In [70]:
#avg. sentiment by year (trends towards 0 due to high # of neutral headlines)
df_clean2 = df_clean.groupby(['publication']).mean().reset_index()
#df_clean2.to_csv('../datasets/news_project/pivot_subjectivity_news.csv')
#df_clean3 = df_clean.groupby(['publish_year','publication']).count().reset_index()
#df_clean3.to_csv('../datasets/news_project/pivot_polarity_news_counts.csv')
#avg. polarity by news org (lots of blank years) https://docs.google.com/spreadsheets/d/1u2ifCTuJPEWx9QkkHOfHwlLA9wJ3YplIURN6Mf_4a8A/edit#gid=1496666001
#raw counts by news org https://docs.google.com/spreadsheets/d/13MXnoLdPlWsWwdHW6bOj80G-NL0oNbyOxaDx-gCk0AQ/edit#gid=962966450

In [72]:
#bring in %s for chart
df_clean2 = df_clean2.sort_values(['SA'],ascending=False)
df_clean2.head(15)

,publication,SA
2,Business Insider,0.060%
12,Vox,0.051%
7,NPR,0.044%
11,Talking Points Memo,0.036%
6,Guardian,0.032%
3,Buzzfeed News,0.028%
0,Atlantic,0.027%
13,Washington Post,0.023%
10,Reuters,0.020%
9,New York Post,0.020%


In [73]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
py.sign_in('username','password')

In [74]:
df_clean2.head(15)

,publication,SA
2,Business Insider,0.060%
12,Vox,0.051%
7,NPR,0.044%
11,Talking Points Memo,0.036%
6,Guardian,0.032%
3,Buzzfeed News,0.028%
0,Atlantic,0.027%
13,Washington Post,0.023%
10,Reuters,0.020%
9,New York Post,0.020%


In [85]:
#bar chart
x = df_clean2['publication']
y = df_clean2['SA']

data = [
    go.Bar(
        x=x,
        y=y,
        marker=dict(
            color='#8bffc0f2',
            line=dict(
                color='#d8ffea96',
                width=1.5
            ),
        ),
        opacity=1
    )
]
layout = go.Layout(
      title='News publications with the most/least positive headlines',
    xaxis=dict(
        title='News publication'
    ),
    yaxis=dict(
        title='Sentiment (higher = more positive',
    ),
    margin=go.Margin(
        l=140,
        r=90,
        b=140,
        t=60,
        pad=4
    ),
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cereal_sunsets')

### % positive/neutral/negative headlines by publication - polarity

In [26]:
#split the SA score into positive/neutral/negative
df_clean['positive'] = np.where(df['SA'] > 0, 1, 0)
df_clean['neutral'] = np.where(df['SA'] == 0, 1, 0)
df_clean['negative'] = np.where(df['SA'] < 0, 1, 0)
df_clean2 = df_clean[['publication','positive','neutral','negative']]
df_clean3 = df_clean2.groupby(['publication']).sum().reset_index()
df_clean3.sort_values(['positive'],ascending=False).head(15)

/Users/erikrood/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/erikrood/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/erikrood/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

,publication,positive,neutral,negative
1,Breitbart,5469,13536,4701
9,New York Post,4677,9196,3573
7,NPR,3451,6359,2010
13,Washington Post,3175,5555,2348
2,Business Insider,2585,2852,1320
10,Reuters,2423,6600,1664
6,Guardian,2366,4558,1700
12,Vox,1730,2134,1014
4,CNN,1591,5056,1279
0,Atlantic,1435,4781,963


In [25]:
#fields for positive/neutral/negative percents
df_clean3['positive_perc'] = \
df_clean3['positive']/(df_clean3['positive']+df_clean3['neutral']+df_clean3['negative'])
df_clean3['neutral_perc'] = \
df_clean3['neutral']/(df_clean3['positive']+df_clean3['neutral']+df_clean3['negative'])
df_clean3['negative_perc'] = \
df_clean3['negative']/(df_clean3['positive']+df_clean3['neutral']+df_clean3['negative'])
#sort and preview dataframe
df_clean3.sort_values(['positive_perc'],ascending=False).head(15)

,publication,positive,neutral,negative,positive_perc,neutral_perc,negative_perc
2,Business Insider,2585,2852,1320,0.382566,0.422081,0.195353
12,Vox,1730,2134,1014,0.354654,0.437474,0.207872
7,NPR,3451,6359,2010,0.291963,0.537986,0.170051
13,Washington Post,3175,5555,2348,0.286604,0.501444,0.211952
3,Buzzfeed News,1354,2549,951,0.278945,0.525134,0.195921
6,Guardian,2366,4558,1700,0.274351,0.528525,0.197124
9,New York Post,4677,9196,3573,0.268084,0.527112,0.204803
11,Talking Points Memo,670,1499,393,0.261514,0.585090,0.153396
1,Breitbart,5469,13536,4701,0.230701,0.570995,0.198304
5,Fox News,990,2493,839,0.229061,0.576816,0.194123


### % of subjective/objective headlines by publication - subjectivity

In [109]:
df = pd.read_csv('../datasets/news_project/all_news_sentiment_raw_subjectivity.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
#extract clean year from date
df[["date"]] = df[["date"]].astype(str) 
df['publish_year'] = df['date'].str[:4]
#filter to 2016, 2017
years = ['2016','2017']
df = df.loc[df['publish_year'].isin(years)]
#NY times looks wonky (skewed positive, dropping from exercise)
df = df.loc[df['publication'] != 'New York Times']
#slice the df to limit to just sentiment analysis score, publication
df_clean = df[['SA', 'publication']]
#pivot to get average subjectivity
df_clean2 = df_clean.groupby(['publication']).mean().reset_index()
df_clean2.head()

,publication,SA
0,Atlantic,0.188560
1,Breitbart,0.234866
2,Business Insider,0.340784
3,Buzzfeed News,0.256399
4,CNN,0.193377


In [110]:
df_clean2.head(15)

,publication,SA
0,Atlantic,0.188560
1,Breitbart,0.234866
2,Business Insider,0.340784
3,Buzzfeed News,0.256399
4,CNN,0.193377
5,Fox News,0.222289
6,Guardian,0.260898
7,NPR,0.249417
8,National Review,0.196043
9,New York Post,0.271212


In [111]:
df_clean['subjective'] = np.where(df['SA'] > .5, 1, 0)
df_clean['objective'] = np.where(df['SA'] <= .5, 1, 0)

/Users/erikrood/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/erikrood/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [112]:
df_clean.head()

,SA,publication,subjective,objective
7799,0.541667,Breitbart,1,0
7800,0.000000,Breitbart,0,1
7801,0.200000,Breitbart,0,1
7802,0.454545,Breitbart,0,1
7803,0.050000,Breitbart,0,1


In [113]:
df_clean2 = df_clean[['publication','subjective','objective']]
df_clean3 = df_clean2.groupby(['publication']).sum().reset_index()
df_clean3.head(15)

,publication,subjective,objective
0,Atlantic,1018,6161
1,Breitbart,4219,19487
2,Business Insider,1896,4861
3,Buzzfeed News,961,3893
4,CNN,1107,6819
5,Fox News,684,3638
6,Guardian,1731,6893
7,NPR,2162,9658
8,National Review,1017,5154
9,New York Post,3838,13608


In [83]:
df_clean3.to_csv('../datasets/news_project/pivot_polarity_news_n_counts.csv')